In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import pycbc
from ler import LeR

In [3]:
ler = LeR(nsamples_mtot=200,
        nsamples_mass_ratio=500,)

Given: IMR waveform, {self.waveform_approximant}.
psds not given. Choosing bilby's default psds
given psds:  {'L1': 'aLIGO_O4_high_asd.txt', 'H1': 'aLIGO_O4_high_asd.txt', 'V1': 'AdV_asd.txt'}
Interpolator will be loaded for L1 detector from ./interpolator_pickle/L1/halfSNR_dict_2.pickle
Interpolator will be loaded for H1 detector from ./interpolator_pickle/H1/halfSNR_dict_2.pickle
Interpolator will be loaded for V1 detector from ./interpolator_pickle/V1/halfSNR_dict_2.pickle


In [4]:
# sampling unlensed events (gw source parameters )
#ler.batch_size=50000
#ler.unlensed_cbc_statistics(nsamples=1000000, json_file='./gw_params_1M.json');
ler.batch_size=25000
ler.unlensed_cbc_statistics(nsamples=100000, json_file='./gw_params_100k.json');

chosen batch size = 25000. If you want to change batch size, self.batch_size = new_size
There will be 4 batche(s)
Batch no. 1
sampling gw source params...
calculating snrs...
Batch no. 2
sampling gw source params...
calculating snrs...
Batch no. 3
sampling gw source params...
calculating snrs...
Batch no. 4
sampling gw source params...
calculating snrs...


In [5]:
# rates
"""unlensed_rate = ler.unlensed_rate(gw_param='./gw_params_1M.json',
    snr_threshold=8.0,
    jsonfile='./gw_params_detectable_1M.json',)"""
unlensed_rate = ler.unlensed_rate(gw_param='./gw_params_100k.json',
    snr_threshold=8.0,
    jsonfile='./gw_params_detectable_100k.json',)

getting gw_params from json file ./gw_params_100k.json...
storing detectable unlensed params in ./gw_params_detectable_100k.json
total unlensed rate with step function: 745.8929572230393
total unlensed rate with pdet function: 817.7646802329512


In [6]:
# sampling lens parameters and image(s) parameters
lensed_param = ler.lensed_cbc_statistics(nsamples=100000, json_file='./lensed_params_100k.json');

chosen batch size = 25000. If you want to change batch size, self.batch_size = new_size
There will be 4 batche(s)
Batch no. 1
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 25000/25000 [00:05<00:00, 4701.04it/s]


calculating snrs...
Batch no. 2
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 25000/25000 [00:05<00:00, 4627.10it/s]


calculating snrs...
Batch no. 3
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 25000/25000 [00:05<00:00, 4534.38it/s]


calculating snrs...
Batch no. 4
sampling lensed params...
solving lens equations...


100%|███████████████████████████████████████████████████████| 25000/25000 [00:05<00:00, 4705.95it/s]


calculating snrs...


## 2 or more image case

### When all image srns are combine

In [7]:
# read json file for lensed_param
with open('./lensed_params_100k.json') as f:
    lensed_param = json.load(f)

In [9]:
# Dimensions are (nsamples, n_max_images)
snr = np.array(lensed_param["opt_snr_net"])
size = len(snr)
snr_threshold = 8.

# if nan in snr, set snr to 0
snr[np.isnan(snr)] = 0.

# squared and add up snr in each row, and then take sqrt
snr = snr**2
snr_sum = np.sum(snr, axis=1)
snr_sum = np.sqrt(snr_sum)

# boolean array for snr_sum > snr_threshold
snr_hit = snr_sum > snr_threshold

# rejection sample wrt to weights
weights = lensed_param["weights"]
not_rejected = np.random.uniform(0, 1, size) < weights
snr_hit = snr_hit & not_rejected

# calculating rates
c0 = ler.lens_galaxy_pop.normalization_pdf_z
total_rate_step = c0 * np.mean(snr_hit)

print("total rate step: ", total_rate_step)


total rate step:  4.435610020767286


### When only two of the snrs are considered seperately

In [10]:
# lensed rates
lensed_rate = ler.lensed_rate(lensed_param='./lensed_params_100k.json',
    jsonfile='./lensed_params_detectable_100k.json',)

getting lensed_param from json file ./lensed_params_100k.json...
storing detectable lensed params in ./lensed_params_detectable_100k.json...
total lensed rate with step function: 0.9382357038477389
total lensed rate with pdet function: 0.9331544587703197


## 4 image case

### When all image srns are combine

In [11]:
# selecting 4 image case
mag = np.array(lensed_param['magnifications'])
snr = np.array(lensed_param["opt_snr_net"])
# checking if each row of snr contains nan
snr_nan = np.isnan(mag)
# sum each row
snr_nan_sum = np.sum(snr_nan, axis=1)
# boolean array for snr_nan_sum == 0
snr_4image = snr_nan_sum == 0

# selection
snr = snr[snr_4image]

In [14]:
# for 100k samples
snr_threshold = 8.
# squared and add up snr in each row, and then take sqrt
snr = snr**2
snr_sum = np.sum(snr, axis=1)
snr_sum = np.sqrt(snr_sum)

# boolean array for snr_sum > snr_threshold
snr_hit = snr_sum > snr_threshold

# rejection sample wrt to weights
weights = np.array(lensed_param["weights"])[snr_4image]
not_rejected = np.random.uniform(0, 1, len(weights)) < weights
snr_hit = snr_hit & not_rejected

print("total snr hits: ", np.sum(snr_hit))

total snr hits:  6502


In [15]:
# for 1M samples
# squared and add up snr in each row, and then take sqrt
"""snr = snr**2
snr_sum = np.sum(snr, axis=1)
snr_sum = np.sqrt(snr_sum)

# boolean array for snr_sum > snr_threshold
snr_hit = snr_sum > snr_threshold

# rejection sample wrt to weights
weights = lensed_param["weights"][snr_4image]
not_rejected = np.random.uniform(0, 1, len(weights)) < weights
snr_hit = snr_hit & not_rejected

print("total snr hits: ", np.sum(snr_hit))"""

total snr hits:  64426


### When only 4 of the snrs are considered seperately

In [19]:
# for 100k samples
# Dimensions are (nsamples, n_max_images)
snr = np.array(lensed_param["opt_snr_net"])
snr = np.array(snr[snr_4image])
size = len(snr)
snr_threshold = 8.

# if nan in snr, set snr to 0
snr[np.isnan(snr)] = 0.

# if the highest and second highest are greater than 8, then set snr as True
"""snr_hit = np.full(len(snr), True)
snr = np.sort(snr, axis=1)
snr_hit = snr_hit & (snr[:, -1] > snr_threshold) & (snr[:, -2] > snr_threshold)"""
snr_hit = snr>8
snr_hit = np.sum(snr_hit,axis=1)

# rejection sample wrt to weights
weights = np.array(lensed_param["weights"])[snr_4image]
not_rejected = np.random.uniform(0, 1, size) < weights
snr_hit = snr_hit & not_rejected

print("total snr hits: ", np.sum(snr_hit))

total snr hits:  553


In [25]:
snr = np.array(lensed_param["opt_snr_net"])
snr = np.array(snr[snr_4image])
size = len(snr)
snr_threshold = 8.

# if nan in snr, set snr to 0
snr[np.isnan(snr)] = 0.

In [16]:
# for 1M samples
# Dimensions are (nsamples, n_max_images)
"""snr = lensed_param["opt_snr_net"]
snr = snr[snr_4image]
size = len(snr)
snr_threshold = 8.

# if nan in snr, set snr to 0
snr[np.isnan(snr)] = 0.

# if the highest and second highest are greater than 8, then set snr as True
#snr_hit = np.full(len(snr), True)
#snr = np.sort(snr, axis=1)
#snr_hit = snr_hit & (snr[:, -1] > snr_threshold) & (snr[:, -2] > snr_threshold)
snr_hit = snr>8
snr_hit = np.sum(snr_hit,axis=1)

# rejection sample wrt to weights
weights = lensed_param["weights"][snr_4image]
not_rejected = np.random.uniform(0, 1, size) < weights
snr_hit = snr_hit & not_rejected

print("total snr hits: ", np.sum(snr_hit))"""


total snr hits:  4582


In [18]:
# for 100k samples
6502/455

14.29010989010989

In [17]:
# for 1M samples
64426/4582

14.060672195547795